In [1]:
import requests
import os
import bs4 
from time import sleep

It's time to recursively solve this mf.

In [2]:
http_request = requests.get("https://github.com/SharmaLlama/ticktack/tree/main/src/data/datasets") 
html_parser = bs4.BeautifulSoup(http_request.text, "html.parser")
github_repos = html_parser.find_all("a", class_="js-navigation-open Link--primary")

In [9]:
data_repos = []
for github_repo in github_repos:
    for js_string in str(github_repo).split('"'):
        if js_string.startswith("/"):
            data_repos.append("".join(["https://github.com", js_string]))

In [14]:
data_set = requests.get(data_repos[0])
passed_data = bs4.BeautifulSoup(data_set.text, "html.parser")
passed_data.find_all("a", class_="js-navigation-open Link--primary")